## 1️⃣ Install Dependencies

In [ ]:
!pip install -q PySastrawi beautifulsoup4 lxml requests pandas numpy scikit-learn

## 2️⃣ Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# NLP
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("✅ Libraries imported successfully!")

## 3️⃣ Daftar Saham Indonesia

In [ ]:
# Daftar Saham IHSG Populer
STOCKS_DATA = {
    # Banking
    'BBCA': {'nama': 'Bank Central Asia', 'sektor': 'Keuangan', 'index': 'LQ45,IDX30'},
    'BBRI': {'nama': 'Bank Rakyat Indonesia', 'sektor': 'Keuangan', 'index': 'LQ45,IDX30'},
    'BMRI': {'nama': 'Bank Mandiri', 'sektor': 'Keuangan', 'index': 'LQ45,IDX30'},
    'BBNI': {'nama': 'Bank Negara Indonesia', 'sektor': 'Keuangan', 'index': 'LQ45,IDX30'},
    'BRIS': {'nama': 'Bank Syariah Indonesia', 'sektor': 'Keuangan', 'index': 'LQ45'},
    
    # Telekomunikasi
    'TLKM': {'nama': 'Telkom Indonesia', 'sektor': 'Telekomunikasi', 'index': 'LQ45,IDX30'},
    'EXCL': {'nama': 'XL Axiata', 'sektor': 'Telekomunikasi', 'index': 'LQ45'},
    'ISAT': {'nama': 'Indosat Ooredoo', 'sektor': 'Telekomunikasi', 'index': 'LQ45'},
    
    # Consumer
    'UNVR': {'nama': 'Unilever Indonesia', 'sektor': 'Consumer Goods', 'index': 'LQ45,IDX30'},
    'ICBP': {'nama': 'Indofood CBP', 'sektor': 'Consumer Goods', 'index': 'LQ45,IDX30'},
    'INDF': {'nama': 'Indofood Sukses Makmur', 'sektor': 'Consumer Goods', 'index': 'LQ45'},
    'MYOR': {'nama': 'Mayora Indah', 'sektor': 'Consumer Goods', 'index': 'LQ45'},
    
    # Mining & Energy
    'ADRO': {'nama': 'Adaro Energy', 'sektor': 'Pertambangan', 'index': 'LQ45,IDX30'},
    'PTBA': {'nama': 'Bukit Asam', 'sektor': 'Pertambangan', 'index': 'LQ45,IDX30'},
    'ITMG': {'nama': 'Indo Tambangraya Megah', 'sektor': 'Pertambangan', 'index': 'LQ45'},
    'ANTM': {'nama': 'Aneka Tambang', 'sektor': 'Pertambangan', 'index': 'LQ45,IDX30'},
    'INCO': {'nama': 'Vale Indonesia', 'sektor': 'Pertambangan', 'index': 'LQ45'},
    'MDKA': {'nama': 'Merdeka Copper Gold', 'sektor': 'Pertambangan', 'index': 'LQ45'},
    
    # Automotive
    'ASII': {'nama': 'Astra International', 'sektor': 'Otomotif', 'index': 'LQ45,IDX30'},
    'AUTO': {'nama': 'Astra Otoparts', 'sektor': 'Otomotif', 'index': 'LQ45'},
    
    # Property
    'BSDE': {'nama': 'Bumi Serpong Damai', 'sektor': 'Properti', 'index': 'LQ45'},
    'CTRA': {'nama': 'Ciputra Development', 'sektor': 'Properti', 'index': 'LQ45'},
    'SMRA': {'nama': 'Summarecon Agung', 'sektor': 'Properti', 'index': 'LQ45'},
    
    # Cement & Construction
    'SMGR': {'nama': 'Semen Indonesia', 'sektor': 'Konstruksi', 'index': 'LQ45'},
    'INTP': {'nama': 'Indocement Tunggal Prakarsa', 'sektor': 'Konstruksi', 'index': 'LQ45'},
    'WIKA': {'nama': 'Wijaya Karya', 'sektor': 'Konstruksi', 'index': 'LQ45'},
    
    # Tech & E-commerce
    'GOTO': {'nama': 'GoTo Gojek Tokopedia', 'sektor': 'Teknologi', 'index': 'LQ45'},
    'BUKA': {'nama': 'Bukalapak', 'sektor': 'Teknologi', 'index': 'IHSG'},
    
    # Healthcare
    'KLBF': {'nama': 'Kalbe Farma', 'sektor': 'Kesehatan', 'index': 'LQ45,IDX30'},
    'SIDO': {'nama': 'Industri Jamu Sido Muncul', 'sektor': 'Kesehatan', 'index': 'LQ45'},
}

# Convert to DataFrame
stocks_df = pd.DataFrame([
    {'kode': k, 'nama': v['nama'], 'sektor': v['sektor'], 'index': v['index']}
    for k, v in STOCKS_DATA.items()
])

print(f"📊 Total {len(stocks_df)} saham tersedia")
stocks_df.head(10)

## 4️⃣ Web Scraper untuk Berita Saham

In [ ]:
class NewsScraper:
    """Scraper untuk mengambil berita saham dari berbagai sumber"""
    
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        self.session = requests.Session()
        self.session.headers.update(self.headers)
    
    def scrape_kontan(self, stock_code: str, max_articles: int = 5) -> List[Dict]:
        """Scrape berita dari Kontan.co.id"""
        articles = []
        search_url = f"https://www.kontan.co.id/search/?search={stock_code}"
        
        try:
            response = self.session.get(search_url, timeout=15)
            soup = BeautifulSoup(response.text, 'lxml')
            
            # Find article links
            links = soup.find_all('a', href=True)
            article_urls = []
            
            for link in links:
                href = link.get('href', '')
                if '/news/' in href or '/investasi/' in href:
                    if href.startswith('http'):
                        article_urls.append(href)
                    elif href.startswith('/'):
                        article_urls.append(f"https://www.kontan.co.id{href}")
            
            article_urls = list(set(article_urls))[:max_articles]
            
            for url in article_urls:
                try:
                    article = self._parse_kontan_article(url)
                    if article:
                        article['stock_code'] = stock_code
                        articles.append(article)
                    time.sleep(1)
                except:
                    continue
                    
        except Exception as e:
            print(f"  ⚠️ Error scraping Kontan for {stock_code}: {e}")
        
        return articles
    
    def _parse_kontan_article(self, url: str) -> Optional[Dict]:
        """Parse artikel Kontan"""
        try:
            response = self.session.get(url, timeout=15)
            soup = BeautifulSoup(response.text, 'lxml')
            
            # Title
            title_tag = soup.find('h1') or soup.find('title')
            title = title_tag.get_text(strip=True) if title_tag else ''
            
            # Content
            content_div = soup.find('div', class_='tmpt-desk') or soup.find('article')
            if content_div:
                paragraphs = content_div.find_all('p')
                content = ' '.join([p.get_text(strip=True) for p in paragraphs])
            else:
                content = ''
            
            if title and len(content) > 100:
                return {
                    'title': title,
                    'content': content[:2000],
                    'url': url,
                    'source': 'Kontan',
                    'scraped_at': datetime.now()
                }
        except:
            pass
        return None
    
    def scrape_detik(self, stock_code: str, max_articles: int = 5) -> List[Dict]:
        """Scrape berita dari Detik Finance"""
        articles = []
        search_url = f"https://www.detik.com/search/searchall?query={stock_code}&siteid=3"
        
        try:
            response = self.session.get(search_url, timeout=15)
            soup = BeautifulSoup(response.text, 'lxml')
            
            # Find articles
            article_items = soup.find_all('article')[:max_articles]
            
            for item in article_items:
                try:
                    link = item.find('a', href=True)
                    if link:
                        url = link.get('href')
                        title = link.get_text(strip=True)
                        
                        if title and url:
                            articles.append({
                                'title': title,
                                'content': title,  # Use title as content for search results
                                'url': url,
                                'source': 'Detik',
                                'stock_code': stock_code,
                                'scraped_at': datetime.now()
                            })
                except:
                    continue
                    
        except Exception as e:
            print(f"  ⚠️ Error scraping Detik for {stock_code}: {e}")
        
        return articles
    
    def scrape_all(self, stock_code: str, max_per_source: int = 3) -> List[Dict]:
        """Scrape dari semua sumber"""
        all_articles = []
        
        # Kontan
        kontan_articles = self.scrape_kontan(stock_code, max_per_source)
        all_articles.extend(kontan_articles)
        
        # Detik
        detik_articles = self.scrape_detik(stock_code, max_per_source)
        all_articles.extend(detik_articles)
        
        return all_articles


# Initialize scraper
scraper = NewsScraper()
print("✅ News Scraper ready!")

## 5️⃣ NLP - Text Preprocessing & Sentiment Analysis

In [ ]:
class IndonesianNLP:
    """NLP Processing untuk Bahasa Indonesia"""
    
    def __init__(self):
        # Stemmer
        factory = StemmerFactory()
        self.stemmer = factory.createStemmer()
        
        # Stopwords
        stop_factory = StopWordRemoverFactory()
        self.stopword_remover = stop_factory.createStopWordRemover()
        
        # Sentiment Lexicon
        self.positive_words = {
            'naik', 'untung', 'profit', 'laba', 'positif', 'bagus', 'baik',
            'meningkat', 'tumbuh', 'berkembang', 'sukses', 'berhasil', 'optimal',
            'kuat', 'solid', 'stabil', 'prospek', 'potensi', 'peluang', 'rebound',
            'bullish', 'outperform', 'buy', 'beli', 'akumulasi', 'dividen',
            'ekspansi', 'inovasi', 'efisien', 'produktif', 'surplus', 'recovery',
            'pulih', 'membaik', 'menguat', 'melesat', 'melonjak', 'meroket',
            'tertinggi', 'rekor', 'fantastis', 'spektakuler', 'cemerlang',
            'moncer', 'ciamik', 'oke', 'mantap', 'jos', 'top', 'keren'
        }
        
        self.negative_words = {
            'turun', 'rugi', 'loss', 'negatif', 'buruk', 'jelek', 'anjlok',
            'menurun', 'merosot', 'jatuh', 'gagal', 'lemah', 'lesu', 'loyo',
            'bearish', 'underperform', 'sell', 'jual', 'defisit', 'krisis',
            'resesi', 'kontraksi', 'penurunan', 'kemerosotan', 'kerugian',
            'koreksi', 'tertekan', 'melemah', 'terpuruk', 'kolaps', 'bangkrut',
            'default', 'utang', 'beban', 'risiko', 'ancaman', 'masalah',
            'kendala', 'hambatan', 'tantangan', 'sulit', 'berat', 'parah',
            'terendah', 'minimum', 'minim', 'seret', 'stagnan', 'mandek'
        }
        
        self.intensifiers = {'sangat', 'sekali', 'amat', 'paling', 'lebih', 'makin', 'semakin', 'begitu', 'terlalu'}
        self.negations = {'tidak', 'bukan', 'belum', 'tanpa', 'jangan', 'tak', 'tiada'}
    
    def preprocess(self, text: str) -> str:
        """Preprocess text"""
        if not text:
            return ""
        
        # Lowercase
        text = text.lower()
        
        # Remove URLs
        text = re.sub(r'http\S+|www\S+', '', text)
        
        # Remove special characters
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # Remove extra whitespace
        text = ' '.join(text.split())
        
        # Remove stopwords
        text = self.stopword_remover.remove(text)
        
        # Stemming
        text = self.stemmer.stem(text)
        
        return text
    
    def analyze_sentiment(self, text: str) -> Dict:
        """Analyze sentiment of text"""
        if not text:
            return {'score': 0, 'label': 'netral', 'positive': 0, 'negative': 0}
        
        text_lower = text.lower()
        words = text_lower.split()
        
        positive_count = 0
        negative_count = 0
        
        for i, word in enumerate(words):
            # Check negation
            is_negated = False
            if i > 0 and words[i-1] in self.negations:
                is_negated = True
            
            # Check intensifier
            intensity = 1.0
            if i > 0 and words[i-1] in self.intensifiers:
                intensity = 1.5
            
            # Count sentiment
            if word in self.positive_words:
                if is_negated:
                    negative_count += intensity
                else:
                    positive_count += intensity
            elif word in self.negative_words:
                if is_negated:
                    positive_count += intensity
                else:
                    negative_count += intensity
        
        # Calculate score
        total = positive_count + negative_count
        if total == 0:
            score = 0
        else:
            score = (positive_count - negative_count) / total
        
        # Determine label
        if score > 0.1:
            label = 'positif'
        elif score < -0.1:
            label = 'negatif'
        else:
            label = 'netral'
        
        return {
            'score': round(score, 3),
            'label': label,
            'positive': positive_count,
            'negative': negative_count
        }


# Initialize NLP
nlp = IndonesianNLP()
print("✅ Indonesian NLP ready!")

## 6️⃣ Content-Based Recommendation Engine

In [ ]:
class StockRecommender:
    """Content-Based Stock Recommendation System"""
    
    def __init__(self, nlp_processor: IndonesianNLP):
        self.nlp = nlp_processor
        self.vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
        self.stock_profiles = {}
        self.news_data = {}
    
    def add_news(self, stock_code: str, articles: List[Dict]):
        """Add news articles for a stock"""
        if stock_code not in self.news_data:
            self.news_data[stock_code] = []
        
        for article in articles:
            # Analyze sentiment
            full_text = f"{article.get('title', '')} {article.get('content', '')}"
            sentiment = self.nlp.analyze_sentiment(full_text)
            
            article['sentiment'] = sentiment
            article['processed_text'] = self.nlp.preprocess(full_text)
            
            self.news_data[stock_code].append(article)
    
    def build_stock_profile(self, stock_code: str) -> Dict:
        """Build profile for a stock based on news"""
        if stock_code not in self.news_data or not self.news_data[stock_code]:
            return None
        
        articles = self.news_data[stock_code]
        
        # Aggregate sentiment
        sentiments = [a['sentiment']['score'] for a in articles]
        avg_sentiment = np.mean(sentiments) if sentiments else 0
        
        # Count labels
        labels = [a['sentiment']['label'] for a in articles]
        positive_count = labels.count('positif')
        negative_count = labels.count('negatif')
        neutral_count = labels.count('netral')
        
        # Combine all text
        combined_text = ' '.join([a['processed_text'] for a in articles if a['processed_text']])
        
        # Calculate recommendation score (0-100)
        sentiment_score = (avg_sentiment + 1) / 2 * 50  # 0-50
        volume_score = min(len(articles) * 5, 30)  # 0-30
        positive_ratio = positive_count / len(articles) * 20 if articles else 0  # 0-20
        
        recommendation_score = sentiment_score + volume_score + positive_ratio
        
        # Determine label
        if recommendation_score >= 70:
            rec_label = 'Strong Buy'
        elif recommendation_score >= 55:
            rec_label = 'Buy'
        elif recommendation_score >= 45:
            rec_label = 'Hold'
        elif recommendation_score >= 30:
            rec_label = 'Sell'
        else:
            rec_label = 'Strong Sell'
        
        profile = {
            'stock_code': stock_code,
            'avg_sentiment': round(avg_sentiment, 3),
            'positive_count': positive_count,
            'negative_count': negative_count,
            'neutral_count': neutral_count,
            'total_news': len(articles),
            'recommendation_score': round(recommendation_score, 2),
            'recommendation_label': rec_label,
            'combined_text': combined_text
        }
        
        self.stock_profiles[stock_code] = profile
        return profile
    
    def find_similar_stocks(self, stock_code: str, top_n: int = 5) -> List[Tuple[str, float]]:
        """Find similar stocks based on news content"""
        if stock_code not in self.stock_profiles:
            return []
        
        # Get all profiles with text
        profiles = [(k, v['combined_text']) for k, v in self.stock_profiles.items() if v['combined_text']]
        
        if len(profiles) < 2:
            return []
        
        codes, texts = zip(*profiles)
        
        # TF-IDF Vectorization
        try:
            tfidf_matrix = self.vectorizer.fit_transform(texts)
        except:
            return []
        
        # Find index of target stock
        try:
            target_idx = codes.index(stock_code)
        except ValueError:
            return []
        
        # Calculate cosine similarity
        similarities = cosine_similarity(tfidf_matrix[target_idx:target_idx+1], tfidf_matrix)[0]
        
        # Get top similar (excluding self)
        similar_indices = similarities.argsort()[::-1][1:top_n+1]
        
        results = [(codes[i], round(similarities[i], 3)) for i in similar_indices if similarities[i] > 0]
        
        return results
    
    def get_recommendations(self) -> pd.DataFrame:
        """Get all stock recommendations sorted by score"""
        if not self.stock_profiles:
            return pd.DataFrame()
        
        data = []
        for code, profile in self.stock_profiles.items():
            stock_info = STOCKS_DATA.get(code, {})
            data.append({
                'Kode': code,
                'Nama': stock_info.get('nama', ''),
                'Sektor': stock_info.get('sektor', ''),
                'Skor': profile['recommendation_score'],
                'Rekomendasi': profile['recommendation_label'],
                'Avg Sentiment': profile['avg_sentiment'],
                'Berita (+)': profile['positive_count'],
                'Berita (-)': profile['negative_count'],
                'Total Berita': profile['total_news']
            })
        
        df = pd.DataFrame(data)
        df = df.sort_values('Skor', ascending=False).reset_index(drop=True)
        
        return df


# Initialize recommender
recommender = StockRecommender(nlp)
print("✅ Stock Recommender ready!")

---
## 7️⃣ Jalankan Analisis

Pilih saham yang ingin dianalisis:

In [ ]:
# Pilih saham untuk dianalisis
# Ubah list ini sesuai kebutuhan

SELECTED_STOCKS = ['BBCA', 'BBRI', 'TLKM', 'ASII', 'ADRO', 'GOTO', 'ANTM', 'KLBF']

print(f"🎯 Saham yang akan dianalisis: {SELECTED_STOCKS}")
print(f"📰 Memulai scraping berita...\n")

# Scrape news for each stock
for stock in SELECTED_STOCKS:
    print(f"📥 Scraping berita untuk {stock}...")
    articles = scraper.scrape_all(stock, max_per_source=3)
    
    if articles:
        recommender.add_news(stock, articles)
        recommender.build_stock_profile(stock)
        print(f"   ✅ {len(articles)} artikel ditemukan")
    else:
        print(f"   ⚠️ Tidak ada artikel ditemukan")
    
    time.sleep(2)  # Delay untuk menghindari rate limiting

print("\n✅ Scraping selesai!")

## 8️⃣ Hasil Rekomendasi

In [ ]:
# Tampilkan hasil rekomendasi
recommendations = recommender.get_recommendations()

if not recommendations.empty:
    print("📊 HASIL REKOMENDASI SAHAM")
    print("=" * 80)
    display(recommendations)
else:
    print("⚠️ Belum ada data untuk ditampilkan")

In [ ]:
# Visualisasi Rekomendasi
import matplotlib.pyplot as plt

if not recommendations.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart - Recommendation Score
    colors = ['#2ecc71' if x >= 55 else '#f39c12' if x >= 45 else '#e74c3c' 
              for x in recommendations['Skor']]
    axes[0].barh(recommendations['Kode'], recommendations['Skor'], color=colors)
    axes[0].set_xlabel('Skor Rekomendasi')
    axes[0].set_title('📈 Skor Rekomendasi Saham')
    axes[0].axvline(x=55, color='green', linestyle='--', alpha=0.5, label='Buy Threshold')
    axes[0].axvline(x=45, color='red', linestyle='--', alpha=0.5, label='Sell Threshold')
    axes[0].legend()
    
    # Bar chart - Sentiment
    colors_sent = ['#2ecc71' if x > 0 else '#e74c3c' if x < 0 else '#95a5a6' 
                   for x in recommendations['Avg Sentiment']]
    axes[1].barh(recommendations['Kode'], recommendations['Avg Sentiment'], color=colors_sent)
    axes[1].set_xlabel('Rata-rata Sentimen')
    axes[1].set_title('💬 Sentimen Berita')
    axes[1].axvline(x=0, color='gray', linestyle='-', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

## 9️⃣ Analisis Detail per Saham

In [ ]:
# Pilih saham untuk analisis detail
DETAIL_STOCK = 'BBCA'  # Ubah sesuai kebutuhan

if DETAIL_STOCK in recommender.stock_profiles:
    profile = recommender.stock_profiles[DETAIL_STOCK]
    stock_info = STOCKS_DATA.get(DETAIL_STOCK, {})
    
    print(f"\n{'='*60}")
    print(f"📊 ANALISIS DETAIL: {DETAIL_STOCK}")
    print(f"{'='*60}")
    print(f"Nama       : {stock_info.get('nama', '-')}")
    print(f"Sektor     : {stock_info.get('sektor', '-')}")
    print(f"Index      : {stock_info.get('index', '-')}")
    print(f"{'='*60}")
    print(f"\n📈 HASIL ANALISIS:")
    print(f"   Skor Rekomendasi : {profile['recommendation_score']}")
    print(f"   Label            : {profile['recommendation_label']}")
    print(f"   Avg Sentiment    : {profile['avg_sentiment']}")
    print(f"\n📰 STATISTIK BERITA:")
    print(f"   Total Berita  : {profile['total_news']}")
    print(f"   Positif       : {profile['positive_count']}")
    print(f"   Negatif       : {profile['negative_count']}")
    print(f"   Netral        : {profile['neutral_count']}")
    
    # Show similar stocks
    similar = recommender.find_similar_stocks(DETAIL_STOCK, top_n=3)
    if similar:
        print(f"\n🔗 SAHAM SERUPA:")
        for code, score in similar:
            print(f"   - {code}: {score:.1%} similarity")
    
    # Show news headlines
    if DETAIL_STOCK in recommender.news_data:
        print(f"\n📰 HEADLINE BERITA:")
        for i, article in enumerate(recommender.news_data[DETAIL_STOCK][:5], 1):
            sentiment = article['sentiment']
            emoji = '🟢' if sentiment['label'] == 'positif' else '🔴' if sentiment['label'] == 'negatif' else '⚪'
            print(f"   {i}. {emoji} {article['title'][:70]}...")
            print(f"      Sumber: {article['source']} | Sentimen: {sentiment['score']}")
else:
    print(f"⚠️ Data untuk {DETAIL_STOCK} tidak tersedia")

## 🔟 Cari Saham Serupa (Content-Based Filtering)

In [ ]:
# Cari saham serupa berdasarkan konten berita
TARGET_STOCK = 'BBCA'  # Ubah sesuai kebutuhan

similar_stocks = recommender.find_similar_stocks(TARGET_STOCK, top_n=5)

if similar_stocks:
    print(f"\n🔍 Saham yang mirip dengan {TARGET_STOCK}:")
    print("-" * 40)
    for code, similarity in similar_stocks:
        stock_info = STOCKS_DATA.get(code, {})
        profile = recommender.stock_profiles.get(code, {})
        print(f"  {code}: {stock_info.get('nama', '')}")
        print(f"     Similarity: {similarity:.1%}")
        print(f"     Skor: {profile.get('recommendation_score', 0)} ({profile.get('recommendation_label', '-')})")
        print()
else:
    print(f"⚠️ Tidak cukup data untuk mencari saham serupa")

---
## 📝 Catatan

- **Skor Rekomendasi** dihitung berdasarkan:
  - Rata-rata sentimen berita
  - Volume berita yang tersedia
  - Rasio berita positif

- **Kategori Rekomendasi:**
  - 🟢 Strong Buy: Skor ≥ 70
  - 🟢 Buy: Skor ≥ 55
  - 🟡 Hold: Skor ≥ 45
  - 🔴 Sell: Skor ≥ 30
  - 🔴 Strong Sell: Skor < 30

- **Disclaimer:** Sistem ini hanya untuk edukasi dan referensi. Bukan rekomendasi investasi finansial.

---
© 2025 Stock Recommendation System